## import

In [1]:
import pylizard, pandas

## parameters

In [2]:
export = [['28EP0026',1],
          ['28EP0088',1],
          ['28EP0088',2],
          ['28FP0029',1],
          ['28FP0029',2],
          ['28FP0033',1],
          ['28FP0033',2],
          ['28FP0067',1],
          ['28FP0067',2],
          ['28FP0094',1],
          ['28FP0094',2],
          ['28FP0131',1],
          ['28FP0133',1],
          ['28FP0133',2],
          ['28FP0273',1],
          ['28FP0273',2],
          ['28FP0273',3],
          ['28FP9002',1],
          ['28FP9014',1],]

yr_min, yr_max = 2016, 2017
xlsx_name = 'export_lizard.xlsx'

## validate
Wanneer een stijghoogte een grotere afstand heeft van de gemiddelde stijghoogte dan n=4 keer de standaardafwijking wordt deze gemarkeerd als outlier.

In [3]:
def validate(h, n=4):
    h_val = h.loc[abs(h-h.mean())<n*h.std()]
    h_out = h.loc[abs(h-h.mean())>=n*h.std()]
    return h_val, h_out

## excel
Loop over alle peilfilters. Per peilfilters wordt er een aparte sheet aangemaakt en worden er grafieken toegevoegd.

In [5]:
writer = pandas.ExcelWriter('export_lizard.xlsx', engine='xlsxwriter') 
for f in export:
    buis, filt = f[0], f[1]
    print(buis, filt)
    
    #Verzamelen peilbuisgegevens, valideren
    p = pylizard.Peilbuis(buis, filt)
    h_val, h_out = validate(p.head_diver)
    h = pandas.concat([h_val, p.head_hand, h_out], axis=1)
    h = h.loc[(h.index.year>=yr_min)&(h.index.year<=yr_max)]
    h.columns = ['head (data logger)', 'head (hand measurement)', 'outlier']
    
    #Aanmaken excel 
    sheet_name = '{}_{}'.format(buis, filt)
    h.to_excel(writer, sheet_name=sheet_name)

    # Toevoegen grafiek
    workbook = writer.book 
    worksheet = writer.sheets[sheet_name]

    chart = workbook.add_chart({'type': 'scatter'})
    chart.add_series({'name':[sheet_name, 0, 1], 
                      'categories': [sheet_name, 1, 0, len(h), 0], 
                      'values':[sheet_name, 1, 1, len(h), 1], 
                      'line':{'color': 'blue'},
                      'marker':{'type': 'none'},})
    chart.add_series({'name':[sheet_name, 0, 2], 
                      'categories': [sheet_name, 1, 0, len(h), 0], 
                      'values':[sheet_name, 1, 2, len(h), 2], 
                      'marker':{'type':'circle', 'border':{'none':True}, 'fill':{'color':'green'}, 'size': 7},})
    chart.add_series({'name':[sheet_name, 0, 3], 
                      'categories': [sheet_name, 1, 0, len(h), 0], 
                      'values':[sheet_name, 1, 3, len(h), 3], 
                      'marker':{'type': 'square', 'border':{'none':True}, 'fill':{'color':'red'}, 'size': 7},})

    chart.set_x_axis({'name': 'Date',
                      'num_format': 'mm-yyyy',
                      'major_gridlines': {'visible': True,}})
    chart.set_y_axis({'name': 'Head',
                      'num_format': '0.00',})

    chart.set_size({'x_scale': 3, 'y_scale': 2})
    worksheet.insert_chart('E2', chart) 
writer.save()

28EP0026 1
28EP0088 1
28EP0088 2
28FP0029 1
28FP0029 2
28FP0033 1
28FP0033 2
28FP0067 1
28FP0067 2
28FP0094 1
28FP0094 2
28FP0131 1
28FP0133 1
28FP0133 2
28FP0273 1
28FP0273 2
28FP0273 3
28FP9002 1
28FP9014 1
